# 向量儲存 Vector datastore


Amazon Aurora 使用 `pgvector`


請注意，

準備好 Aurora 集群
在 Aurora PostgreSQL 資料庫 (DB) 叢集上建立 pgvector 擴充
建立擴展向量；


連接到 Aurora 叢集並檢查

```sql
-- 顯示當前資料庫
SELECT current_database();

-- 顯示資料庫中的資料表
SELECT table_name
FROM postgres.information_schema.tables
WHERE table_schema = 'public';
```


## 導入函式庫


In [1]:
# BedrockEmbeddings 類別是用來將文本轉換成嵌入向量的工具
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores.pgvector import PGVector, DistanceStrategy
import os
from dotenv import load_dotenv

# 載入環境參數
load_dotenv()

True

## Create a `collection` and `connect` with the Vector store


In [2]:
# Collection Name
COLLECTION_NAME = "my_collection"

# Connection String
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.getenv("PGVECTOR_DRIVER"),
    user=os.getenv("PGVECTOR_USER"),
    password=os.getenv("PGVECTOR_PASSWORD"),
    host=os.getenv("PGVECTOR_HOST"),
    port=os.getenv("PGVECTOR_PORT"),
    database=os.getenv("PGVECTOR_DATABASE"),
)

# 創建一個實例，用來將文本轉換成嵌入向量
embeddings = BedrockEmbeddings()

# 建立 VectorDB 儲存實例
my_vector_store = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
    distance_strategy=DistanceStrategy.EUCLIDEAN,
)

/Users/samhsiao/Documents/PythonVenv/envllmChatBot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See https://github.com/langchain-ai/langchain-postgres for details aboutthe new implementation.
  warn_deprecated(
/Users/samhsiao/Documents/PythonVenv/envllmChatBot/lib/python3.11/site-packages/langchain_community/vectorstores/pgvector.py:322: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name co

此時，`LangChain`將在 Aurora 資料庫中建立**兩個**表：

- `langchain_pg_collection`
- `langchain_pg_embedding`

We can see these **two** tables


![Vector Store on AWS](images/sql_lc_show_tables.png)


We can see our newly created `COLLECTION` **my_collection**


![Vector Store on AWS](images/sql_lc_show_tables2.png)


Let's see what we have inside the other table `langchain_pg_embedding`


![Vector Store on AWS](images/sql_lc_show_tables3.png)


## Create some `vectors`


In [8]:
texts = ["新德里是印度的首都", "歡迎來到印度", "我今天要去踢足球"]

# Text --> Embeddings --> Vectors --> Aurora
my_vector_store.from_texts(
    texts=texts,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding=embeddings,
)

Let's now check the `langchain_pg_embedding` table


![Vector Store on AWS](images/sql_lc_show_tables4.png)


#### Add few more vectors


In [12]:
texts = [
    "今晚的天空晴朗",
    "貓是好奇的動物",
    "巴黎正在下雨",
    "學習 Python 可以很有趣",
    "和朋友一起喝咖啡味道更好",
    "我住在波士頓，這是一個美麗的城市",
    "我家旁邊有一個博物館",
    "音樂將人們聚集在一起",
    "博物館在少數地方週一休館",
    "很少有地方博物館每週開放 7 天，就像在我的城市一樣"
]

# Text --> Embeddings --> Vectors --> Aurora
my_vector_store.from_texts(
    texts=texts,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding=embeddings,
)

## 使用 `PGVector` 進行 `相似度搜尋`


In [13]:
my_vector_store.similarity_search(
    query="任何城市的博物館都是全天開放嗎",
    # 從搜索結果中返回最相似的4個結果
    k=4
)

[Document(page_content='很少有地方博物館每週開放 7 天，就像在我的城市一樣'),
 Document(page_content='很少有地方博物館每週開放 7 天，就像在我的城市一樣'),
 Document(page_content='博物館在少數地方週一休館'),
 Document(page_content='博物館在少數地方週一休館')]